In [ ]:
### import libraries###
%pylab inline
import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score

import tensorflow as tf
import keras
import imageio

In [ ]:
### set seed value for control model randomness###
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed) # seed for make random number predictable

In [ ]:
### set dir path###
root_dir = os.path.abspath('../..')
data_dir = os.path.join(root_dir, 'Desktop', 'identify_the_digit_analyitcs_vidhya') # os.path.join(path, 'folder', 'next folder')
images_dir = os.path.join(root_dir, 'Desktop', 'images')
# check for existence
os.path.exists(root_dir)
os.path.exists(data_dir)
os.path.exists(images_dir)


In [ ]:
###loading data and Preprocessing###
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'Test.csv'))
sample_submission = pd.read_csv(os.path.join(data_dir, 'Sample_Submission.csv'))

train.head()

In [ ]:
### read image data###
img_name = rng.choice(train.filename)
filepath = os.path.join(images_dir, 'train', img_name)

img = imageio.imread(filepath)

pylab.imshow(img, cmap='gray') # cmap -> color map
pylab.axis('off')
pylab.show()


In [ ]:
# represent image as numpy array
img

In [ ]:
###data manipulation###

### store image as numpy arrays
temp = []
for img_name in train.filename:
    image_path = os.path.join(images_dir, 'train', img_name)
    img = imageio.imread(image_path)
    img = img.astype('float32')
    temp.append(img)
    
train_x = np.stack(temp)

train_x /= 255.0
train_x = train_x.reshape(-1, 784).astype('float32')

temp = []
for img_name in test.filename:
    image_path = os.path.join(images_dir, 'test', img_name)
    img = imageio.imread(image_path)
    img = img.astype('float32')
    temp.append(img)
    
test_x = np.stack(temp)

test_x /= 255.0
test_x = test_x.reshape(-1, 784).astype('float32')
train_y = keras.utils.np_utils.to_categorical(train.label.values)

### As this is a typical ML problem, to test the proper functioning of our model we create a validation set. Let’s take a split size of 70:30 for train set vs validation set
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train_y[:split_size], train_y[split_size:]
train.label.ix[split_size:]

In [ ]:
### model building###

### neural network 3 layer###

# input 28x28 images output 10x1 vector 50 neural in hidden layer use Adam<optimization algorithm>

# define vars
input_num_units = 784 # 28x28 images
hidden_num_units = 50 # neural
output_num_units = 10

epochs = 5
batch_size = 128

# import keras modules

from keras.models import Sequential
from keras.layers import Dense

# create model
model = Sequential([
  Dense(hidden_num_units, kernel_initializer=input_num_units, activation='relu'),
  Dense(output_num_units, kernel_initializer=hidden_num_units, activation='softmax'),
])
# input_dim change to kernel_initializer
# compile the model with necessary attributes
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
### train model###
trained_model = model.fit(train_x, train_y, nb_epoch=epochs, batch_size=batch_size, validation_data=(val_x, val_y))

In [ ]:
### model evaluate###
pred = model.predict_classes(test_x)

img_name = rng.choice(test.filename)
filepath = os.path.join(data_dir, 'Train', 'Images', 'test', img_name)

img = imread(filepath, flatten=True)

test_index = int(img_name.split('.')[0]) - train.shape[0]

print "Prediction is: ", pred[test_index]

pylab.imshow(img, cmap='gray')
pylab.axis('off')
pylab.show()

In [ ]:
### create submission###